# 面向對象(進階)

在前面的章節我們已經了解了面向對象的入門知識，知道瞭如何定義類，如何創建對像以及如何給對象發消息。為了能夠更好的使用面向對象編程思想進行程序開發，我們還需要對Python中的面向對象編程進行更為深入的了解。

## @property装饰器

之前我們討論過Python中屬性和方法訪問權限的問題，雖然我們不建議將屬性設置為私有的，但是如果直接將屬性暴露給外界也是有問題的，比如我們沒有辦法檢查賦給屬性的值是否有效。我們之前的建議是將屬性命名以單下劃線開頭，通過這種方式來暗示屬性是受保護的，不建議外界直接訪問，那麼如果想訪問屬性可以通過屬性的getter（訪問器）和setter（修改器）方法進行對應的操作。如果要做到這點，就可以考慮使用@property包裝器來包裝getter和setter方法，使得對屬性的訪問既安全又方便，代碼如下所示。

In [1]:
class Person(object):

    def __init__(self, name, age):
        self._name = name
        self._age = age

    # 訪問器- getter方法
    @property
    def name(self):
        return self._name

    # 訪問器 - getter方法
    @property
    def age(self):
        return self._age

    # 修改器 - setter方法
    @age.setter
    def age(self, age):
        self._age = age

    def play(self):
        if self._age <= 16:
            print('%s is playing chess.' % self._name)
        else:
            print('%s is playing monopoly.' % self._name)


def main():
    person = Person('Carson', 19)
    person.play()
    person.age = 20
    person.play()
    # person.name = 'Ula'  # AttributeError: can't set attribute


if __name__ == '__main__':
    main()

Carson is playing monopoly.
Carson is playing monopoly.


## _slots__魔法

我們講到這裡，不知道大家是否已經意識到，Python是一門[動態語言](https://zh.wikipedia.org/wiki/动态语言)。通常，動態語言允許我們在程序運行時給對象綁定新的屬性或方法，當然也可以對已經綁定的屬性和方法進行解綁定。但是如果我們需要限定自定義類型的對像只能綁定某些屬性，可以通過在類中定義__slots__變量來進行限定。需要注意的是__slots__的限定只對當前類的對像生效，對子類並不起任何作用。

In [2]:
class Person(object):

    # 限定Person對像只能綁定_name, _age和_gender屬性    __slots__ = ('_name', '_age', '_gender')
    def __init__(self, name, age):
        self._name = name
        self._age = age

    @property
    def name(self):
        return self._name

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):
        self._age = age

    def play(self):
        if self._age <= 16:
            print('%s is playing chess.' % self._name)
        else:
            print('%s is playing monopoly.' % self._name)


def main():
    person = Person('Carson', 19)
    person.play()
    person._gender = 'Girl'
    # AttributeError: 'Person' object has no attribute '_is_gay'
    # person._is_gay = True

### 靜態方法和類方法

之前，我們在類中定義的方法都是對象方法，也就是說這些方法都是發送給對象的消息。實際上，我們寫在類中的方法並不需要都是對象方法，例如我們定義一個“三角形”類，通過傳入三條邊長來構造三角形，並提供計算周長和麵積的方法，但是傳入的三條邊長未必能構造出三角形對象，因此我們可以先寫一個方法來驗證三條邊長是否可以構成三角形，這個方法很顯然就不是對象方法，因為在調用這個方法時三角形對象尚未創建出來（因為都不知道三條邊能不能構成三角形），所以這個方法是屬於三角形類而並不屬於三角形對象的。我們可以使用靜態方法來解決這類問題，代碼如下所示。

In [3]:
from math import sqrt


class Triangle(object):

    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c

    @staticmethod
    def is_valid(a, b, c):
        return a + b > c and b + c > a and a + c > b

    def perimeter(self):
        return self._a + self._b + self._c

    def area(self):
        half = self.perimeter() / 2
        return sqrt(half * (half - self._a) *
                    (half - self._b) * (half - self._c))


def main():
    a, b, c = 3, 4, 5
    # 靜態方法和類方法都是通過各類發消息來調用的
    if Triangle.is_valid(a, b, c):
        t = Triangle(a, b, c)
        print(t.perimeter())
    # 也可以通過各類發消息來調用對象方法但是要傳入接收消息的對像作為參數
    # print(Triangle.perimeter(t))
        print(t.area())
        # print(Triangle.area(t))
    else:
        print('Cannot form a triangle.')


if __name__ == '__main__':
    main()

12
6.0


和靜態方法比較類似，Python還可以在類中定義類方法，類方法的第一個參數約定名為cls，它代表的是當前類相關的信息的對象（類本身也是一個對象，有的地方也稱之為類的元數據對象），通過這個參數我們可以獲取和類相關的信息並且可以創建出類的對象，代碼如下所示。

In [4]:
from time import time, localtime, sleep


class Clock(object):
    """Digital clock"""

    def __init__(self, hour=0, minute=0, second=0):
        self._hour = hour
        self._minute = minute
        self._second = second

    @classmethod
    def now(cls):
        ctime = localtime(time())
        return cls(ctime.tm_hour, ctime.tm_min, ctime.tm_sec)

    def run(self):
        self._second += 1
        if self._second == 60:
            self._second = 0
            self._minute += 1
            if self._minute == 60:
                self._minute = 0
                self._hour += 1
                if self._hour == 24:
                    self._hour = 0

    def show(self):
        """Show Time"""
        return '%02d:%02d:%02d' % \
               (self._hour, self._minute, self._second)


def main():
    # 通過類方法創建對象並獲取系統時間
    clock = Clock.now()
    while True:
        print(clock.show())
        sleep(1)
        clock.run()


if __name__ == '__main__':
    main()

18:51:55
18:51:56
18:51:57
18:51:58
18:51:59
18:52:00
18:52:01
18:52:02


KeyboardInterrupt: 

## 類之間的關係

簡單的說，類和類之間的關係有三種：is-a、has-a和use-a關係。

- is-a關係也叫繼承或泛化，比如學生和人的關係、手機和電子產品的關係都屬於繼承關係。
- has-a關係通常稱之為關聯，比如部門和員工的關係，汽車和引擎的關係都屬於關聯關係；關聯關係如果是整體和部分的關聯，那麼我們稱之為聚合關係；如果整體進一步負責了部分的生命週期（整體和部分是不可分割的，同時同在也同時消亡），那麼這種就是最強的關聯關係，我們稱之為合成關係。
- use-a關係通常稱之為依賴，比如司機有一個駕駛的行為（方法），其中（的參數）使用到了汽車，那麼司機和汽車的關係就是依賴關係。
我們可以使用一種叫做[UML](https://zh.wikipedia.org/wiki/统一建模语言)（統一建模語言）的東西來進行面向對象建模，其中一項重要的工作就是把類和類之間的關係用標準化的圖形符號描述出來。關於UML我們在這裡不做詳細的介紹，有興趣的讀者可以自行閱讀《UML面向對象設計基礎》一書。

![](https://raw.githubusercontent.com/jackfrued/Python-100-Days/master/Day01-15/res/uml-components.png)

![](https://raw.githubusercontent.com/jackfrued/Python-100-Days/master/Day01-15/res/uml-example.png)

利用類之間的這些關係，我們可以在已有類的基礎上來完成某些操作，也可以在已有類的基礎上創建新的類，這些都是實現代碼復用的重要手段。復用現有的代碼不僅可以減少開發的工作量，也有利於代碼的管理和維護，這是我們在日常工作中都會使用到的技術手段。

## 繼承和多態

剛才我們提到了，可以在已有類的基礎上創建新類，這其中的一種做法就是讓一個類從另一個類那裡將屬性和方法直接繼承下來，從而減少重複代碼的編寫。提供繼承信息的我們稱之為父類，也叫超類或基類；得到繼承信息的我們稱之為子類，也叫派生類或衍生類。子類除了繼承父類提供的屬性和方法，還可以定義自己特有的屬性和方法，所以子類比父類擁有的更多的能力，在實際開發中，我們經常會用子類對象去替換掉一個父類對象，這是面向對象編程中一個常見的行為，對應的原則稱之為[里氏替換原則](https://zh.wikipedia.org/wiki/里氏替换原则)。下面我們先看一個繼承的例子。

In [5]:
class Person(object):
    """Person"""

    def __init__(self, name, age):
        self._name = name
        self._age = age

    @property
    def name(self):
        return self._name

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):
        self._age = age

    def play(self):
        print('%s is playing happily.' % self._name)

    def watch_av(self):
        if self._age >= 18:
            print('%sis watching action movie.' % self._name)
        else:
            print('%s can only watch《We Bare Bears 》.' % self._name)


class Student(Person):
    """Student"""

    def __init__(self, name, age, grade):
        super().__init__(name, age)
        self._grade = grade

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, grade):
        self._grade = grade

    def study(self, course):
        print('%s %s is studying %s.' % (self._grade, self._name, course))


class Teacher(Person):
    """Teacher"""

    def __init__(self, name, age, title):
        super().__init__(name, age)
        self._title = title

    @property
    def title(self):
        return self._title

    @title.setter
    def title(self, title):
        self._title = title

    def teach(self, course):
        print('%s %s is talking %s.' % (self._name, self._title, course))


def main():
    stu = Student('Carson', 19, 'freshman')
    stu.study('Physics')
    stu.watch_av()
    t = Teacher('Ula', 22, 'Doctor')
    t.teach('Python Programing')
    t.watch_av()


if __name__ == '__main__':
    main()

freshman Carson is studying Physics.
Carsonis watching action movie.
Ula Doctor is talking Python Programing.
Ulais watching action movie.


子類在繼承了父類的方法後，可以對父類已有的方法給出新的實現版本，這個動作稱之為方法重寫（override）。通過方法重寫我們可以讓父類的同一個行為在子類中擁有不同的實現版本，當我們調用這個經過子類重寫的方法時，不同的子類對象會表現出不同的行為，這個就是多態（poly-morphism）

In [6]:
from abc import ABCMeta, abstractmethod


class Pet(object, metaclass=ABCMeta):
    """Pet"""

    def __init__(self, nickname):
        self._nickname = nickname

    @abstractmethod
    def make_voice(self):
        """Makes a sound"""
        pass


class Dog(Pet):
    """Dog"""

    def make_voice(self):
        print('%s: Wang ~Wang ~' % self._nickname)


class Cat(Pet):
    """Cat"""

    def make_voice(self):
        print('%s: Meow ~Meow ~' % self._nickname)


def main():
    pets = [Dog('Protein'), Cat('Amino acid'), Dog('nerve')]
    for pet in pets:
        pet.make_voice()


if __name__ == '__main__':
    main()

Protein: Wang ~Wang ~
Amino acid: Meow ~Meow ~
nerve: Wang ~Wang ~


在上面的代碼中，我們將Pet類處理成了一個抽像類，所謂抽像類就是不能夠創建對象的類，這種類的存在就是專門為了讓其他類去繼承它。 Python從語法層面並沒有像Java或C#那樣提供對抽像類的支持，但是我們可以通過abc模塊的ABCMeta元類和abstractmethod包裝器來達到抽像類的效果，如果一個類中存在抽象方法那麼這個類就不能夠實例化（創建對象）。上面的代碼中，Dog和Cat兩個子類分別對Pet類中的make_voice抽象方法進行了重寫並給出了不同的實現版本，當我們在main函數中調用該方法時，這個方法就表現出了多態行為（同樣的方法做了不同的事情）。

## 綜合案例

### 案例1：奧特曼打小怪獸。

In [7]:
from abc import ABCMeta, abstractmethod
from random import randint, randrange


class Fighter(object, metaclass=ABCMeta):
    """Fighter"""

    # 通過__slots__魔法限定對象可以綁定的成員變量
    __slots__ = ('_name', '_hp')

    def __init__(self, name, hp):
        """
        Initialization method
        :param name: Name
        :param hp: Life value
        """
        self._name = name
        self._hp = hp

    @property
    def name(self):
        return self._name

    @property
    def hp(self):
        return self._hp

    @hp.setter
    def hp(self, hp):
        self._hp = hp if hp >= 0 else 0

    @property
    def alive(self):
        return self._hp > 0

    @abstractmethod
    def attack(self, other):
        """
        Attack
        :param other: Attacked object
        """
        pass


class Ultraman(Fighter):
    """Ultraman"""

    __slots__ = ('_name', '_hp', '_mp')

    def __init__(self, name, hp, mp):
        """
        Initialization method
        :param name: Name
        :param hp: Life value
        :param mp: Magical value
        """
        super().__init__(name, hp)
        self._mp = mp

    def attack(self, other):
        other.hp -= randint(15, 25)

    def huge_attack(self, other):
        """
        Ultimate killing skills (knock out at least 50 points or three quarters of the opponent's blood)
        :param other: Attacked object

        :return: Return True if successful, otherwise False
        """
        if self._mp >= 50:
            self._mp -= 50
            injury = other.hp * 3 // 4
            injury = injury if injury >= 50 else 50
            other.hp -= injury
            return True
        else:
            self.attack(other)
            return False

    def magic_attack(self, others):
        """
        Magic attack
        :param others: Attacked group

        :return: Use magic to return True otherwise return False
        """
        if self._mp >= 20:
            self._mp -= 20
            for temp in others:
                if temp.alive:
                    temp.hp -= randint(10, 15)
            return True
        else:
            return False

    def resume(self):
        """Restore magical value"""
        incr_point = randint(1, 10)
        self._mp += incr_point
        return incr_point

    def __str__(self):
        return '~~~%sUltraman~~~\n' % self._name + \
            'Life value: %d\n' % self._hp + \
            'Magical value: %d\n' % self._mp


class Monster(Fighter):
    """Little monster"""

    __slots__ = ('_name', '_hp')

    def attack(self, other):
        other.hp -= randint(10, 20)

    def __str__(self):
        return '~~~%sLittle monster~~~\n' % self._name + \
            'Life value: %d\n' % self._hp


def is_any_alive(monsters):
    """Determine if the little monster is alive"""
    for monster in monsters:
        if monster.alive > 0:
            return True
    return False


def select_alive_one(monsters):
    """Select a little living monster"""
    monsters_len = len(monsters)
    while True:
        index = randrange(monsters_len)
        monster = monsters[index]
        if monster.alive > 0:
            return monster


def display_info(ultraman, monsters):
    """Display information about Ultraman and Little Monster"""
    print(ultraman)
    for monster in monsters:
        print(monster, end='')


def main():
    u = Ultraman('Carson', 1000, 120)
    m1 = Monster('Ula', 250)
    m2 = Monster('Peter', 500)
    m3 = Monster('Jackson', 750)
    ms = [m1, m2, m3]
    fight_round = 1
    while u.alive and is_any_alive(ms):
        print('========Round %02d========' % fight_round)
        m = select_alive_one(ms)  # 選中一隻小怪獸
        skill = randint(1, 10)   # 通過隨機數選擇使用哪種技能
        if skill <= 6:  # 60%的概率使用普通攻擊
            print('%s hit with a normal attack %s.' % (u.name, m.name))
            u.attack(m)
            print('%s has %d of magical value restored.' % (u.name, u.resume()))
        elif skill <= 9:  # 30%的概率使用魔法攻击(可能因魔法值不足而失败)
            if u.magic_attack(ms):
                print('%s used magic attack.' % u.name)
            else:
                print('%s failed to use magic.' % u.name)
        else:  # 10%的概率使用究极必杀技(如果魔法值不足则使用普通攻击)
            if u.huge_attack(m):
                print('%sUse the ultimate killing technique to abuse %s.' % (u.name, m.name))
            else:
                print('%s hit with a normal attack %s.' % (u.name, m.name))
                print('%s has %d of magical value restored.' % (u.name, u.resume()))
        if m.alive > 0:  # 如果选中的小怪兽没有死就回击奥特曼
            print('%s hit back %s.' % (m.name, u.name))
            m.attack(u)
        display_info(u, ms)  # 每个回合结束后显示奥特曼和小怪兽的信息
        fight_round += 1
    print('\n========End of battle!========\n')
    if u.alive > 0:
        print('%s Ultraman wins!' % u.name)
    else:
        print('Little monster wins!')


if __name__ == '__main__':
    main()

========Round 01========
Carson used magic attack.
Peter hit back Carson.
~~~CarsonUltraman~~~
Life value: 990
Magical value: 100

~~~UlaLittle monster~~~
Life value: 239
~~~PeterLittle monster~~~
Life value: 490
~~~JacksonLittle monster~~~
Life value: 736
========Round 02========
Carson hit with a normal attack Jackson.
Carson has 8 of magical value restored.
Jackson hit back Carson.
~~~CarsonUltraman~~~
Life value: 972
Magical value: 108

~~~UlaLittle monster~~~
Life value: 239
~~~PeterLittle monster~~~
Life value: 490
~~~JacksonLittle monster~~~
Life value: 712
========Round 03========
Carson hit with a normal attack Peter.
Carson has 9 of magical value restored.
Peter hit back Carson.
~~~CarsonUltraman~~~
Life value: 954
Magical value: 117

~~~UlaLittle monster~~~
Life value: 239
~~~PeterLittle monster~~~
Life value: 467
~~~JacksonLittle monster~~~
Life value: 712
========Round 04========
Carson hit with a normal attack Jackson.
Carson has 9 of magical value restored.
Jackson hit b

### 案例2：撲克遊戲。

In [8]:
import random


class Card(object):
    """A card"""

    def __init__(self, suite, face):
        self._suite = suite
        self._face = face

    @property
    def face(self):
        return self._face

    @property
    def suite(self):
        return self._suite

    def __str__(self):
        if self._face == 1:
            face_str = 'A'
        elif self._face == 11:
            face_str = 'J'
        elif self._face == 12:
            face_str = 'Q'
        elif self._face == 13:
            face_str = 'K'
        else:
            face_str = str(self._face)
        return '%s%s' % (self._suite, face_str)
    
    def __repr__(self):
        return self.__str__()


class Poker(object):
    """a deck of cards"""

    def __init__(self):
        self._cards = [Card(suite, face) 
                       for suite in '♠♥♣♦'
                       for face in range(1, 14)]
        self._current = 0

    @property
    def cards(self):
        return self._cards

    def shuffle(self):
        """Shuffle (random order)"""
        self._current = 0
        random.shuffle(self._cards)

    @property
    def next(self):
        """Licensing"""
        card = self._cards[self._current]
        self._current += 1
        return card

    @property
    def has_next(self):
        """Is there any card?"""
        return self._current < len(self._cards)


class Player(object):
    """Player"""

    def __init__(self, name):
        self._name = name
        self._cards_on_hand = []

    @property
    def name(self):
        return self._name

    @property
    def cards_on_hand(self):
        return self._cards_on_hand

    def get(self, card):
        """Draw"""
        self._cards_on_hand.append(card)

    def arrange(self, card_key):
        """Players sort out the cards in their hands"""
        self._cards_on_hand.sort(key=card_key)


# 排序規則-先根據花色再根據點數排序
def get_key(card):
    return (card.suite, card.face)


def main():
    p = Poker()
    p.shuffle()
    players = [Player('Carson'), Player('Peter'), Player('Ula'), Player('Jackson')]
    for _ in range(13):
        for player in players:
            player.get(p.next)
    for player in players:
        print(player.name + ':', end=' ')
        player.arrange(get_key)
        print(player.cards_on_hand)


if __name__ == '__main__':
    main()

Carson: [♠6, ♣4, ♣5, ♣8, ♣9, ♣Q, ♥A, ♥5, ♦A, ♦3, ♦4, ♦10, ♦Q]
Peter: [♠A, ♠9, ♠K, ♣2, ♣J, ♣K, ♥2, ♥3, ♥4, ♥8, ♥J, ♥K, ♦9]
Ula: [♠2, ♠3, ♠4, ♠7, ♠J, ♣3, ♣7, ♥6, ♥9, ♦2, ♦5, ♦6, ♦8]
Jackson: [♠5, ♠8, ♠10, ♠Q, ♣A, ♣6, ♣10, ♥7, ♥10, ♥Q, ♦7, ♦J, ♦K]


### 案例3：工資結算系統。

In [9]:
"""
某公司有三種類型的員工 分別是部門經理、程序員和銷售員
需要設計一個工資結算系統 根據提供的員工信息來計算月薪
部門經理的月薪是每月固定15000元
程序員的月薪按本月工作時間計算 每小時150元
銷售員的月薪是1200元的底薪加上銷售額5%的提成
"""
from abc import ABCMeta, abstractmethod


class Employee(object, metaclass=ABCMeta):
    """Employee"""

    def __init__(self, name):
        """
        Initialization method
        :param name: Name
        """
        self._name = name

    @property
    def name(self):
        return self._name

    @abstractmethod
    def get_salary(self):
        """
        Get a monthly salary

        :return: Monthly salary
        """
        pass


class Manager(Employee):
    """Department manager"""

    def get_salary(self):
        return 15000.0


class Programmer(Employee):
    """Programmer"""

    def __init__(self, name, working_hour=0):
        super().__init__(name)
        self._working_hour = working_hour

    @property
    def working_hour(self):
        return self._working_hour

    @working_hour.setter
    def working_hour(self, working_hour):
        self._working_hour = working_hour if working_hour > 0 else 0

    def get_salary(self):
        return 150.0 * self._working_hour


class Salesman(Employee):
    """Seller"""

    def __init__(self, name, sales=0):
        super().__init__(name)
        self._sales = sales

    @property
    def sales(self):
        return self._sales

    @sales.setter
    def sales(self, sales):
        self._sales = sales if sales > 0 else 0

    def get_salary(self):
        return 1200.0 + self._sales * 0.05


def main():
    emps = [
        Manager('Evan'), Programmer('Lily'),
        Manager('Hellen'), Salesman('Adele'),
        Salesman('Brianna'), Programmer('Brook'),
        Programmer('Charlotte')
    ]
    for emp in emps:
        if isinstance(emp, Programmer):
            emp.working_hour = int(input('Please enter %s working hours this month: ' % emp.name))
        elif isinstance(emp, Salesman):
            emp.sales = float(input('Please enter %s sales this month: ' % emp.name))
        # 同樣是接收get_salary這個消息但是不同的員工表現出了不同的行為(多態)
        print('%s Salary for this month: $ %s' %
              (emp.name, emp.get_salary()))


if __name__ == '__main__':
    main()

Evan Salary for this month: $ 15000.0
Please enter Lily working hours this month: 30
Lily Salary for this month: $ 4500.0
Hellen Salary for this month: $ 15000.0
Please enter Adele sales this month: 12
Adele Salary for this month: $ 1200.6
Please enter Brianna sales this month: 29
Brianna Salary for this month: $ 1201.45
Please enter Brook working hours this month: 25
Brook Salary for this month: $ 3750.0
Please enter Charlotte working hours this month: 32
Charlotte Salary for this month: $ 4800.0
